## code for supplementary figure 3

In [ ]:
import matplotlib.pyplot as plt
import polars as pl
from sklearn.metrics import roc_curve
import seaborn as sns
import numpy as np

In [ ]:
df_mod4 = pl.read_csv('./mod4_age_edu_mmse_MEGaverage.csv')

In [ ]:
df_mod4

In [ ]:
df_mod4['lower'].abs(), (df_mod4['AME'] - df_mod4['SE'] * 1.96).abs()

In [ ]:
df_mod4['upper'].abs(), (df_mod4['AME'] + df_mod4['SE'] * 1.96).abs()

In [ ]:
df_mod4 = df_mod4.sort(pl.col('AME').abs(), descending=False)

In [ ]:
factors=df_mod4['factor'].to_list()

In [ ]:
roc1 = pl.read_csv('./Biofind_code_R1/roc_data_mod1.csv')
roc2 = pl.read_csv('./Biofind_code_R1/roc_data_mod2.csv')
roc3 = pl.read_csv('./Biofind_code_R1/roc_data_mod3_MEG_average.csv')
roc4 = pl.read_csv('./Biofind_code_R1/roc_data_mod4_MEG_average.csv')
rocs =[roc1, roc2, roc3, roc4]
aucs = [0.71, 0.75, 0.77, 0.79]

In [ ]:
mapping = {
    'age': 'Age',
    'Edu_years': 'Education',
     'MMSE': 'MMSE',
     'ratio_Hippocampus_TotalGrayVol': 'Hippocampal ratio',
     'meg_average_adj': 'MEG average'
}

In [ ]:
colors = plt.cm.viridis(np.linspace(0.1, 0.9, 5))
colors2 = plt.cm.plasma(np.linspace(0.1, 0.95, 5))

sns.set_context('paper', font_scale=1.5)

fig, axes = plt.subplot_mosaic(
    [['roc', 'roc', 'roc', 'blank', 'coef', 'coef', 'coef']]
)
fig.set_size_inches(
    8, 3.5
)

axes['blank'].set_axis_off()

### panel A

ax = axes['roc']
for jj, roc in enumerate(rocs):
    ax.plot(
        1 - roc['specificity'],
        roc['sensitivity'],
        color=colors2[jj],
        linewidth=2.5,
        label=f'Model {jj+1} (AUC={aucs[jj]:0.2f})'
    )

ax.set_xticks(np.arange(0, 1.1, 0.2))
ax.set_yticks(np.arange(0, 1.1, 0.2))
ax.set_xlabel('1 - specificity', loc='center')
ax.set_ylabel('sensitivity', loc='center')
ax.text(-0.3, 1, 'b', fontsize='large', transform=ax.transAxes, weight='bold')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], frameon=False, fontsize=10)
sns.despine(trim=True, ax=ax)

### panel B

ax = axes['coef']
for ii, (factor, ame, se, *_) in enumerate(df_mod4.rows()):
    ax.errorbar(
        x=ame, y=ii, xerr=se * 1.96, fmt='o', capsize=5, capthick=2.5,
        ms=10, linewidth=3, color=colors[ii], mfc='white', mew=3
    )
ax.axvline(0, linestyle='--', color='gray')


mapped_factors = [mapping.get(f, f) for f in factors]
ax.set_yticks(range(len(factors)), mapped_factors)
ax.set_xticks(np.arange(-0.15, 0.10, 0.05))
ax.set_xlabel(r'$\Delta$ Predicted probability of progressing to AD', loc='right')
ax.text(-0.75, 1, 'a', fontsize='large', transform=ax.transAxes, weight='bold')

sns.despine(trim=True, ax=ax)

fig.savefig('suppl_fig3_R3.pdf', dpi=300, bbox_inches='tight')
fig.savefig('suppl_fig3_R3.png', dpi=300,  bbox_inches='tight')